# Grounding Guardrail 

### Fact-Checking

The goal of the self-check fact-checking output rail is to ensure that the answer to a RAG (Retrieval Augmented Generation) query is grounded in the provided evidence extracted from the knowledge base (KB).

NeMo Guardrails uses the concept of **relevant chunks** (which are stored in the `$relevant_chunks` context variable) as the evidence against which fact-checking should be performed. The relevant chunks can be extracted automatically, if the built-in knowledge base support is used, or provided directly alongside the query (see the [Getting Started Guide example](../getting_started/7_rag)).

**IMPORTANT**: The performance of this rail is strongly dependent on the capability of the LLM to follow the instructions in the `self_check_facts` prompt.

In [ ]:
import sys
import os

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [ ]:
import json
import boto3
from utils import bedrock, print_ww
import pandas as pd

# This helper function encompasses the process of initializing NeMo Guardrails and generating Rails based on a specified configuration.
from nemoguardrails import LLMRails, RailsConfig

# BedrockModels is a "Singleton" class which initializes the necessary models for the notebook.
from models import BedrockModels

## Initalize bedrock client

In [ ]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True,
)

## Define function to create a LLMRails instance with Claudev2 as LLM and a certain rail configuration

In [ ]:
# This helper function encapsulates the necessary steps to bootstrap
# NeMo Guardrails and returns Rails based on a given configuration.
def bootstrap_bedrock_nemo_guardrails(rail_config_path: str) -> LLMRails:

    # 1. initialize rails config
    config = RailsConfig.from_path(f"NeMo/rails/{rail_config_path}/config")

    # initialize bedrock models
    # you can pass model id as string or use the default model id 'anthropic.claude-v2'
    bedrock_models = BedrockModels
    bedrock_models.init_bedrock_client(boto3_bedrock)
    bedrock_models.init_llm("anthropic.claude-v2")

    # 2. bootstraps NeMo Guardrails with the necessary resources
    app = LLMRails(config=config, llm=bedrock_models.llm, verbose=True)

    return app

In [ ]:
rails = bootstrap_bedrock_nemo_guardrails("grounding")

# Test the rail

In [ ]:
!cat ./NeMo/rails/grounding/config/factcheck.co

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Enable or disable the _check_facts_ action by setting the boolean variable below

In [ ]:
response = await rails.generate_async(
    messages=[
        {"role": "context", "content": {"check_facts": True}},
        {
            "role": "user",
            "content": "How many vacation days does the ABC company offer?",
        },
    ]
)

print_ww(response["content"])

# Try-Out
#### Try These Moderation Sample Questions

#### <font color="#4ec8b0">Fact Check successful:</font>
* How many vacation days does the ABC company offer?
* What are the policies about confidentiality and intellectual property?

#### <font color="#c84e4e">Fact Check unsuccessful:</font>

*  What is the bereavement leave policy regarding paid time off for family members' loss?"
* What is the notice period at ABC company?
